In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [4]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 10

In [5]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('data/mnist', one_hot=True) 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


In [51]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# Features are of the type float, and labels are of the type int

X = tf.placeholder(tf.float32, shape=[batch_size, 784], name="X")
y = tf.placeholder(tf.int32, shape=[batch_size, 10], name="y")

In [52]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
W = tf.Variable(tf.random_normal(shape=[784, 10], mean=0, stddev=0.1), name="W")
b = tf.Variable(tf.zeros(shape=[1, 10]), name="b")

In [55]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE
logits = tf.matmul(X, W) + b

# This is called the logit function: 
# logit(y) = log(y / (1-y))
#
# logit as parameter is just an unprocessed parameter, in this case unprocessed for 
# softmax and cross entropy

In [56]:
# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch
# TODO: why?
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y, name="entropy")
loss = tf.reduce_mean(entropy)

In [57]:
# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [58]:
lossOptimizer = optimizer.minimize(loss)

In [66]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # TO-DO: run optimizer + fetch loss_batch
            _, loss_batch = sess.run([lossOptimizer, loss], {X:X_batch, y:Y_batch})
            #print(loss_batch)
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch[0]

print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 0.384589884526
Average loss epoch 1: 0.296203943642
Average loss epoch 2: 0.287582972806
Average loss epoch 3: 0.274708461702
Average loss epoch 4: 0.275104753117
Average loss epoch 5: 0.274616294755
Average loss epoch 6: 0.272725915593
Average loss epoch 7: 0.268482098386
Average loss epoch 8: 0.267530814491
Average loss epoch 9: 0.262540632785
Total time: 5.33101797104 seconds
Optimization Finished!
Accuracy 0.9227
